# First assignment

In this notebook we are collecting the necessary data for our data science capstone project. First we scrape the wikipedia page by using requests to retrieve the hmtl which then is transformed into a BeautifulSoup object. From there we extract the extract the needed information and store the data in a pandas dataframe.

First thing to do is to load the necessary packages. Make sure you have install all of them. It happens that you have to install the "lxml" package separately.

In [1]:
# %pip install lxml
# %pip install bs4
# %pip install requests
# %pip install pandas
from bs4 import BeautifulSoup
import pandas as pd
import requests

Now we are ready to get the data from the link and convert it into a BeautifulSoup object.

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_html = requests.get(wiki_url)
soup = BeautifulSoup(raw_html.text)

Firstly we create a list. After finding the table and table data we create a dictionary called cell having 3 keys PostalCode, Borough and Neighborhood. In the end we create a dataframe with the list.

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

df=pd.DataFrame(table_contents)

Let's check what we've got:

In [4]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


This dataframe looks good. But taking a closer look we see that we can clean the 'Borugh' column better. For example we replace 'Downtown TorontoStn A PO Boxes25 The Esplanade' with 'Downtown Toronto Stn A'

In [5]:

df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

A last check that all postal codes are unique and that no cell has a borough but a Not assigned  neighborhood.

In [7]:
print('All postal codes are unique:', len(df.PostalCode.unique())==df.shape[0])

not_assigned_n = False if sum(df.Neighborhood=='Not assigned')==0 else True
print("A cell has a borough but a 'Not assigned' neighborhood:", not_assigned_n)


All postal codes are unique: True
A cell has a borough but a 'Not assigned' neighborhood: False
